In [221]:
import pandas as pd
from bs4 import BeautifulSoup, Comment, NavigableString, Tag
import re
import urllib
import difflib
from nltk import sent_tokenize
import math
import re

In [206]:
df_versions = pd.read_parquet('/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/link_versions.parquet')
df_versions.head(5)

,source_title,target_title,source_ID,target_ID,first_version,second_version,first_text,second_text
0,Andouille,Cane,18,26429,7592354,9074938,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...
1,Andouille,Pecan,18,168446,7592354,9074938,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...
2,Andouille,Taste,18,13407,7592354,9074938,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...,[[File:Andouille.jpg|thumb]]\n[[File:Andouille...
3,Astronomy,Black_hole,48,3506,9035453,9035459,[[File:The Galactic Centre and Bulge above the...,[[File:The Galactic Centre and Bulge above the...
4,Astronomy,Cosmic_ray,48,241923,9035250,9035453,[[File:The Galactic Centre and Bulge above the...,[[File:The Galactic Centre and Bulge above the...


In [207]:
sample = df_versions.sample(1)
source_title = sample['source_title'].iloc[0]
source_ID = sample['source_ID'].iloc[0]
first_version = sample['first_version'].iloc[0]
second_version = sample['second_version'].iloc[0]

In [239]:
file_1 = f'/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/pages/{source_ID}_{first_version}.html'
file_2 = f'/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/pages/{source_ID}_{second_version}.html'
with open(file_1, 'r') as f:
    text_1 = f.read()
with open(file_2, 'r') as f:
    text_2 = f.read()
print(text_2)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>June 21 - Simple English Wikipedia, the free encyclopedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vect

In [240]:
html_1 = BeautifulSoup(text_1, 'html.parser')
html_2 = BeautifulSoup(text_2, 'html.parser')
print(len(html_1.prettify()))
print(len(html_2.prettify()))

356983
368629


In [241]:
def simplify_html(html):
    # only keep the tag with class 'mw-parser-output'
    html = html.find('div', {'class': 'mw-parser-output'})
    # remove all figures, tables, captions, sup, style
    # figures
    for figure in html.find_all('figure'):
        figure.decompose()
    # tables
    for table in html.find_all('table'):
        table.decompose()
    # captions
    for caption in html.find_all('caption'):
        caption.decompose()
    # sup
    for sup in html.find_all('sup'):
        sup.decompose()
    # style
    for style in html.find_all('style'):
        style.decompose()
    
    # remove all comments
    comments = html.find_all(string=lambda text: isinstance(text, Comment))
    [comment.extract() for comment in comments]
    
    # remove all tags with class 'mw-editsection'
    for tag in html.find_all('span', {'class': 'mw-editsection'}):
        tag.decompose()
    
    # remove all tags with class 'metadata'
    for tag in html.find_all('div', {'class': 'metadata'}):
        tag.decompose()
        
    # remove all tags with class 'reflist'
    for tag in html.find_all('div', {'class': 'reflist'}):
        tag.decompose()
        
    # remove all links with class 'external text'
    for tag in html.find_all('a', {'class': 'external text'}):
        tag.decompose()
        
    # remove all map tags
    for tag in html.find_all('map'):
        tag.decompose()
    return html

In [242]:
html_1_clean = simplify_html(html_1)
html_2_clean = simplify_html(html_2)
text_1 = "\n".join([line for line in html_1_clean.prettify().split('\n') if line.strip() != ''])
text_2 = "\n".join([line for line in html_2_clean.prettify().split('\n') if line.strip() != ''])
sentences_1 = sent_tokenize(text_1)
sentences_2 = sent_tokenize(text_2)

In [243]:
# for each sentence, if it is less than 50 characters, merge it with the next and previous sentences
i = 0
while i < len(sentences_1):
    if len(sentences_1[i]) < 50:
        if i > 0 and i < len(sentences_1) - 1:
            sentences_1[i-1] = sentences_1[i-1] + ' ' + sentences_1[i] + ' ' + sentences_1[i+1]
            del sentences_1[i]
            del sentences_1[i]
        elif i == 0:
            sentences_1[i] = sentences_1[i] + ' ' + sentences_1[i+1]
            del sentences_1[i+1]
        else:
            sentences_1[i-1] = sentences_1[i-1] + ' ' + sentences_1[i]
            del sentences_1[i]
    else:
        i += 1

i = 0
while i < len(sentences_2):
    if len(sentences_2[i]) < 50:
        if i > 0 and i < len(sentences_2) - 1:
            sentences_2[i-1] = sentences_2[i-1] + ' ' + sentences_2[i] + ' ' + sentences_2[i+1]
            del sentences_2[i]
            del sentences_2[i]
        elif i == 0:
            sentences_2[i] = sentences_2[i] + ' ' + sentences_2[i+1]
            del sentences_2[i+1]
        else:
            sentences_2[i-1] = sentences_2[i-1] + ' ' + sentences_2[i]
            del sentences_2[i]
    else:
        i += 1
        

In [244]:
# dont allow for links to be separated across sentences
i = 0
while i < len(sentences_1):
    while sentences_1[i].count('<a') > sentences_1[i].count('</a>'):
        sentences_1[i] = sentences_1[i] + ' ' + sentences_1[i+1]
        del sentences_1[i+1]
    i += 1

i = 0
while i < len(sentences_2):
    while sentences_2[i].count('<a') > sentences_2[i].count('</a>'):
        sentences_2[i] = sentences_2[i] + ' ' + sentences_2[i+1]
        del sentences_2[i+1]
    i += 1


In [245]:
# dont allow parenthesis to be separated across sentences
i = 0
while i < len(sentences_1) - 1:
    # find right-most occurrence of ')' and '(' in sentences_1[i]
    right_paren_before = sentences_1[i].rfind(')')
    left_paren_before = sentences_1[i].rfind('(')
    right_paren_before = right_paren_before if right_paren_before != -1 else 0
    left_paren_before = left_paren_before if left_paren_before != -1 else 0
    # find left-most occurrence of ')' and '(' in sentences_1[i+1]
    right_paren_after = sentences_1[i+1].find(')')
    left_paren_after = sentences_1[i+1].find('(')
    right_parent_after = right_paren_after if right_paren_after != -1 else len(sentences_1[i+1])
    left_parent_after = left_paren_after if left_paren_after != -1 else len(sentences_1[i+1])

    if right_paren_before < left_paren_before and right_parent_after < left_paren_after:
        sentences_1[i] = sentences_1[i] + ' ' + sentences_1[i+1]
        del sentences_1[i+1]
    else:
        i += 1

i = 0
while i < len(sentences_2) - 1:
    # find right-most occurrence of ')' and '(' in sentences_2[i]
    right_paren_before = sentences_2[i].rfind(')')
    left_paren_before = sentences_2[i].rfind('(')
    right_paren_before = right_paren_before if right_paren_before != -1 else 0
    left_paren_before = left_paren_before if left_paren_before != -1 else 0
    # find left-most occurrence of ')' and '(' in sentences_2[i+1]
    right_paren_after = sentences_2[i+1].find(')')
    left_paren_after = sentences_2[i+1].find('(')
    right_parent_after = right_paren_after if right_paren_after != -1 else len(sentences_2[i+1])
    left_parent_after = left_paren_after if left_paren_after != -1 else len(sentences_2[i+1])

    if right_paren_before < left_paren_before and right_parent_after < left_paren_after:
        sentences_2[i] = sentences_2[i] + ' ' + sentences_2[i+1]
        del sentences_2[i+1]
    else:
        i += 1 

In [ ]:
# dont allow list items to be separated across sentences
i = 0
while i < len(sentences_1) - 1:
    while sentences_1[i].count('<li') > sentences_1[i].count('</li>'):
        sentences_1[i] = sentences_1[i] + ' ' + sentences_1[i+1]
        del sentences_1[i+1]
    i += 1

i = 0
while i < len(sentences_2) - 1:
    while sentences_2[i].count('<li') > sentences_2[i].count('</li>'):
        sentences_2[i] = sentences_2[i] + ' ' + sentences_2[i+1]
        del sentences_2[i+1]
    i += 1

In [246]:
# force </li> to act as a separator
i = 0
while i < len(sentences_1):
    if '</li>' in sentences_1[i]:
        extra_sentences = [s.strip() + '</li>' for s in sentences_1[i].split('</li>') if s.strip() != '']
        del sentences_1[i]
        sentences_1 = sentences_1[:i] + extra_sentences + sentences_1[i:]
    i += 1
    
i = 0
while i < len(sentences_2):
    if '</li>' in sentences_2[i]:
        extra_sentences = [s.strip() + '</li>' for s in sentences_2[i].split('</li>') if s.strip() != '']
        del sentences_2[i]
        sentences_2 = sentences_2[:i] + extra_sentences + sentences_2[i:]
    i += 1        

In [247]:
# print the differences between the two versions
d = difflib.Differ()

In [248]:
print(source_title)
print(sample['target_title'].iloc[0])
print(first_version)
print(second_version)

June_21
Writer
7803953
9079583


In [249]:
diff = d.compare(sentences_1, sentences_2)
for line in diff:
    if line.startswith('?'):
        continue
    print('###')
    print(line)

###
  <div class="mw-parser-output">
 <p>
  <b>
   June 21
  </b>
  is the 172nd day of the year (173rd in
  <a href="/wiki/Leap_year" title="Leap year">
   leap years
  </a>
  ) in the
  <a href="/wiki/Gregorian_calendar" title="Gregorian calendar">
   Gregorian calendar
  </a>
  .
###
  There are 193 days remaining until the end of the year.
###
  In most years, this is the date of the
  <a href="/wiki/Summer" title="Summer">
   summer
  </a>
  <a href="/wiki/Solstice" title="Solstice">
   solstice
  </a>
  in the
  <a class="mw-redirect" href="/wiki/Northern_hemisphere" title="Northern hemisphere">
   northern hemisphere
  </a>
  , as the day with the most daylight hours.
###
  In the
  <a href="/wiki/Southern_hemisphere" title="Southern hemisphere">
   southern hemisphere
  </a>
  , this is the
  <a href="/wiki/Winter" title="Winter">
   winter
  </a>
  <a href="/wiki/Solstice" title="Solstice">
   solstice
  </a>
  , with the least daylight hours.
###
  </p>
 <meta property="mw:Pa

In [250]:
df_versions[df_versions['source_title'] == 'Nazi_Germany']

,source_title,target_title,source_ID,target_ID,first_version,second_version,first_text,second_text
804,Nazi_Germany,Adolf_Hitler,15678,4457,9077857,9077860,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
805,Nazi_Germany,Adolf_Hitler,15678,4457,9077857,9077860,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
806,Nazi_Germany,Africa,15678,1942,9077857,9077860,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
807,Nazi_Germany,Africa,15678,1942,9077857,9077860,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
808,Nazi_Germany,Afrika_Korps,15678,300589,9077857,9077860,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
...,...,...,...,...,...,...,...,...
1279,Nazi_Germany,White_people,15678,271006,9033893,9033894,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
1280,Nazi_Germany,Winston_Churchill,15678,8471,9033893,9033894,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
1281,Nazi_Germany,Winston_Churchill,15678,8471,9033893,9033894,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...
1282,Nazi_Germany,World_War_II,15678,219837,9033893,9033894,<br />{{big|{{nobold|''Deutsches Reich''}}}}}}...,'''Nazi Germany''' (officially known as the ''...


Match original and modified elements

In [251]:
original_sentences = []
new_sentences = []
diff = d.compare(sentences_1, sentences_2)
for line in diff:
    if line.startswith('?'):
        continue
    elif line.startswith('+'):
        new_sentences.append({'added': True, 'index': len(new_sentences), 'match': None, 'sentence': line[2:].strip()})
        words = new_sentences[-1]['sentence'].split()
        freqs = {}
        for word in words:
            freqs[word] = freqs.get(word, 0) + 1
        norm = math.sqrt(sum([freqs[word] ** 2 for word in freqs]))
        best_match = {'index': None, 'score': 0}
        for i, sentence in enumerate(original_sentences):
            # compute cossine similarity between all the sentence, and get the highest score
            words = sentence['sentence'].split()
            freqs2 = {}
            for word in words:
                freqs2[word] = freqs2.get(word, 0) + 1
            norm2 = math.sqrt(sum([freqs2[word] ** 2 for word in freqs2]))
            score = 0
            for word in freqs:
                score += freqs[word] * freqs2.get(word, 0)
            score = score / (norm * norm2)
            if score > best_match['score']:
                best_match['index'] = i
                best_match['score'] = score
        print(best_match)
        if best_match['score'] > 0.75:
            original_sentences[best_match['index']]['match'] = new_sentences[-1]['index']
            new_sentences[-1]['match'] = original_sentences[best_match['index']]['index']
    elif line.startswith('-'):
        original_sentences.append({'removed': True, 'index': len(original_sentences), 'match': None, 'sentence': line[2:].strip()})
    else:
        original_sentences.append({'removed': False, 'index': len(original_sentences), 'match': None, 'sentence': line.strip()})
        new_sentences.append({'added': False, 'index': len(new_sentences), 'match': None, 'sentence': line.strip()})

{'index': 52, 'score': 0.9999791001208892}
{'index': 53, 'score': 0.9999878151599206}


In [252]:
for sentence in original_sentences:
    print(sentence)
print('#########')
for sentence in new_sentences:
    print(sentence)

{'removed': False, 'index': 0, 'match': None, 'sentence': '<div class="mw-parser-output">\n <p>\n  <b>\n   June 21\n  </b>\n  is the 172nd day of the year (173rd in\n  <a href="/wiki/Leap_year" title="Leap year">\n   leap years\n  </a>\n  ) in the\n  <a href="/wiki/Gregorian_calendar" title="Gregorian calendar">\n   Gregorian calendar\n  </a>\n  .'}
{'removed': False, 'index': 1, 'match': None, 'sentence': 'There are 193 days remaining until the end of the year.'}
{'removed': False, 'index': 2, 'match': None, 'sentence': 'In most years, this is the date of the\n  <a href="/wiki/Summer" title="Summer">\n   summer\n  </a>\n  <a href="/wiki/Solstice" title="Solstice">\n   solstice\n  </a>\n  in the\n  <a class="mw-redirect" href="/wiki/Northern_hemisphere" title="Northern hemisphere">\n   northern hemisphere\n  </a>\n  , as the day with the most daylight hours.'}
{'removed': False, 'index': 3, 'match': None, 'sentence': 'In the\n  <a href="/wiki/Southern_hemisphere" title="Southern hemisp